In [1]:
import pandas as pd
import pickle
import numpy as np
import json
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString

import timeit

In [163]:
acc = pd.read_pickle('data/accident.pickle')

In [83]:
acc.shape

(200183, 33)

In [105]:
# New Year's Eve crash map

#fig, ax = plt.subplots(figsize = (8, 10))
#acc.plot(ax = ax, alpha = 0.4, color = 'grey')
#acc[(acc['month'] == 12) & (acc['day']== 31)].plot(ax = ax, markersize =20, color = 'red', marker = 'o', label = 'Collision')


In [106]:
# Fatality crash map (red) vs property only damage crashes (blue) map
#fig, ax = plt.subplots(figsize = (8, 10))
#acc.plot(ax = ax, alpha = 0.4, color = 'grey', markersize = 0.1)
#acc[acc['SEVERITYDESC']=='Fatality Collision'].plot(ax = ax, markersize =30, color = 'red', marker = 'o', label = 'Collision')
#acc[acc['SEVERITYDESC']=='Property Damage Only Collision'].plot(ax = ax, markersize = .01, color = 'blue', marker = 'o')

In [240]:
volume = gpd.read_file('https://opendata.arcgis.com/datasets/170b764c52f34c9497720c0463f3b58b_9.geojson')


In [245]:
volume.rename(columns={'OBJECTID': 'SEGID'},inplace=True)

In [41]:
roads = gpd.read_file('https://opendata.arcgis.com/datasets/383027d103f042499693da22d72d10e3_0.geojson')

In [84]:
len(acc['FIRST_STNAME_ORD'].unique())

499

In [85]:
len(volume['FIRST_STNAME_ORD'].unique())

501

In [165]:
acc.head()

,OBJECTID,ADDRTYPE,LOCATION,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,...,time,year,month,day,dayofweek,hour,FIRST_STNAME_ORD,AWDT_ROUND,DOWNTOWN,SEGID
0,1,Intersection,GREEN LAKE WAY N AND N 46TH ST,1,Property Damage Only Collision,Angles,2,0,0,2,...,2018-03-26 12:50:00,2018,3,26,0,12,NE 50TH ST,23700.0,N,519
1,2,Block,LAKE CITY WAY NE BETWEEN NE 125TH ST AND NE 12...,1,Property Damage Only Collision,Other,1,0,0,1,...,2018-03-23 03:30:00,2018,3,23,4,3,NE 145TH ST,38000.0,N,96
2,3,Block,1ST AVE NE BETWEEN NE 50TH ST AND NE 51ST ST,1,Property Damage Only Collision,Parked Car,2,0,0,2,...,2018-03-24 15:00:00,2018,3,24,5,15,NE 50TH ST,24600.0,N,518
3,4,Block,25TH AVE NE BETWEEN NE BLAKELEY ST AND NE 54TH ST,1,Property Damage Only Collision,Parked Car,2,0,0,2,...,2018-03-22 17:14:00,2018,3,22,3,17,25TH AVE NE,18000.0,N,467
4,5,Block,N 36TH ST BETWEEN PHINNEY AVE N AND FRANCIS AVE N,1,Property Damage Only Collision,Parked Car,2,0,0,2,...,2018-03-25 02:03:00,2018,3,25,6,2,NW 36TH ST,22600.0,N,615


In [211]:
#s = acc.groupby(['year', 'SEGID'])['AWDT_ROUND'].sum()

In [235]:
#volume = s.loc[[2016]]
#volume

In [247]:
y16 = acc[acc['year']==2016]

In [251]:
# total traffic volume by road segment for 2016 
count16 = y16.groupby('SEGID')['AWDT_ROUND'].count().to_frame(name = 'count')


In [252]:
volume = volume.merge(count16, on = 'SEGID')

In [253]:
volume['rate'] = volume['count']/(volume['AWDT_ROUND']*365)


In [262]:
y = np.random.randint(1,60, size = 1653)


5.0228310502283106e-05

In [ ]:
# nearest road blocks to each road segment
dist = []
for segment in volume.geometry:
    d = []
    for block in roads.geometry:
        d.append(segment.distance(block))
    d = np.array(d)
    dist.append(d.argmin())

In [246]:
volume

,SEGID,COUNT_COMPKEY,FIRST_STNAME_ORD,FIRST_ARTDESCRIPT,FIRST_TRANDESCRIPT,COUNTFLAG,FLOWSEGID,DOWNTOWN,COUNTSTART,COUNTAADT,COUNTAAWDT,ESTIMATE,AWDT_ROUND,SHAPE_Length,geometry
0,1,None,S SPOKANE ST,None,None,Y,2019.0,N,2015-04-03T00:00:00,5508.833000,6829.504000,None,6800.0,1118.471355,LINESTRING (-122.3396800681195 47.571700326672...
1,2,None,BATTERY ST TUNNEL NB,None,None,Y,2023.0,N,2015-06-09T00:00:00,27080.835000,28793.392000,None,28800.0,1605.935119,LINESTRING (-122.3392266031084 47.570739990412...
2,3,None,ALASKAN WY VI SB,None,None,Y,2024.0,N,2015-06-09T00:00:00,0.000000,23393.664000,None,23400.0,2441.189454,LINESTRING (-122.3394359390524 47.566345148425...
3,4,None,S SPOKANE ST RP,None,None,Y,2025.0,N,2015-06-09T00:00:00,3622.684000,4421.927000,None,4400.0,1214.542393,LINESTRING (-122.3434333843779 47.571579040153...
4,5,None,WEST SEATTLE BR WB,None,None,Y,2026.0,N,2014-04-29T00:00:00,46875.085335,50036.029406,None,50000.0,1817.352489,LINESTRING (-122.333137393517 47.5715774100566...
5,6,None,WEST SEATTLE BR EB,None,None,Y,2027.0,N,None,NaN,35000.000000,Y,35000.0,1423.838986,LINESTRING (-122.3309157221583 47.571518426658...
6,7,None,W SEA BR EB 4 AV OFF RP,None,None,Y,2028.0,N,None,NaN,24000.000000,Y,24000.0,1458.971527,LINESTRING (-122.3309157221583 47.571518426658...
7,8,None,MYERS WAY S,None,None,Y,2029.0,N,None,NaN,38000.000000,Y,38000.0,335.499087,LINESTRING (-122.3348413540749 47.522322332560...
8,9,None,S CLOVERDALE ST,None,None,Y,2030.0,N,None,NaN,13000.000000,Y,13000.0,987.546181,LINESTRING (-122.3281987581469 47.526481143795...
9,10,None,S CLOVERDALE ST,None,None,Y,2031.0,N,None,NaN,11000.000000,Y,11000.0,1305.323633,LINESTRING (-122.3229156126081 47.526465474329...


In [1]:
acc

NameError: name 'acc' is not defined